In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1C/blp25_hatespeech_subtask_1C_dev.tsv
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1C/blp25_hatespeech_subtask_1C_dev_test.tsv
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1C/blp25_hatespeech_subtask_1C_train.tsv
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install torch
!pip install scikit-learn
# !pip install --upgrade accelerate
import logging
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional
import pandas as pd
import datasets
import evaluate
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict
import torch
from sklearn.model_selection import StratifiedKFold

import transformers
from transformers import (
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version
from transformers.modeling_outputs import SequenceClassifierOutput


logger = logging.getLogger(__name__)

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)
train_file = 'blp25_hatespeech_subtask_1C_train.tsv'
validation_file = 'blp25_hatespeech_subtask_1C_dev.tsv'
test_file = 'blp25_hatespeech_subtask_1C_dev_test.tsv'
import os
os.environ["WANDB_DISABLED"] = "true"
training_args = TrainingArguments(
    learning_rate=2e-5,
    num_train_epochs=1,
    per_device_train_batch_size=8,  # Reduced batch size for GAT
    per_device_eval_batch_size=8,   # Reduced batch size
    output_dir="./distilBERT_m/",
    overwrite_output_dir=True,
    remove_unused_columns=False,
    local_rank= 1,
    load_best_model_at_end=True,
    save_total_limit=2,
    save_strategy="no",
    report_to=None,
    gradient_accumulation_steps=2,  # Use gradient accumulation to maintain effective batch size
    fp16=True,  # Use mixed precision training to save memory
)

max_train_samples = None
max_eval_samples=None
max_predict_samples=None
max_seq_length = 512
batch_size = 16
transformers.utils.logging.set_verbosity_info()

log_level = training_args.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f" distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
logger.info(f"Training/evaluation parameters {training_args}")
model_name = 'csebuetnlp/banglabert'
set_seed(training_args.seed)
hate_type_map = {'None': 0, 'Religious Hate': 1, 'Sexism': 2, 'Political Hate': 3, 'Profane': 4, 'Abusive': 5}
severity_map = {'Little to None': 0, 'Mild': 1, 'Severe': 2}
to_whom_map = {'None': 0, 'Individual': 1, 'Organization': 2, 'Community': 3, 'Society': 4}
id2hate = {v: k for k, v in hate_type_map.items()}
id2sev = {v: k for k, v in severity_map.items()}
id2to = {v: k for k, v in to_whom_map.items()}

# Load training and validation data
train_df = pd.read_csv(train_file, sep='\t')

train_df['hate_type'] = train_df['hate_type'].fillna('None')
train_df['to_whom'] = train_df['to_whom'].fillna('None')
train_df['hate_type'] = train_df['hate_type'].map(hate_type_map).astype(int)
train_df['hate_severity'] = train_df['hate_severity'].map(severity_map).astype(int)
train_df['to_whom'] = train_df['to_whom'].map(to_whom_map).astype(int)

validation_df = pd.read_csv(validation_file, sep='\t')
validation_df['hate_type'] = validation_df['hate_type'].replace('nan', 'None').fillna('None')
validation_df['to_whom'] = validation_df['to_whom'].replace('nan', 'None').fillna('None')
validation_df['hate_type'] = validation_df['hate_type'].map(hate_type_map).astype(int)
validation_df['hate_severity'] = validation_df['hate_severity'].map(severity_map).astype(int)
validation_df['to_whom'] = validation_df['to_whom'].map(to_whom_map).astype(int)

# # Combine training and validation data for cross-validation
combined_df = pd.concat([train_df, validation_df], ignore_index=True)
combined_dataset = Dataset.from_pandas(combined_df)

# # Load test data separately
test_df = pd.read_csv(test_file, sep='\t')
test_dataset = Dataset.from_pandas(test_df)

# Create initial dataset dict for preprocessing
raw_datasets = DatasetDict({
    "combined": combined_dataset,
    "test": test_dataset
})

for key in raw_datasets.keys():
    logger.info(f"loading a local file for {key}")
    
print(f"Combined dataset size: {len(combined_dataset)}")
print(f"Test dataset size: {len(test_df)}")
len(test_df['id'])
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=None,
    use_fast=True,
    revision="main",
    use_auth_token=None,
)

class MultiTaskModel(torch.nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.base_model = AutoModel.from_pretrained(model_name)
        self.config = AutoConfig.from_pretrained(model_name)
        hidden_size = self.config.hidden_size
        self.hate_type_head = torch.nn.Linear(hidden_size, len(hate_type_map))
        self.severity_head = torch.nn.Linear(hidden_size, len(severity_map))
        self.to_whom_head = torch.nn.Linear(hidden_size, len(to_whom_map))

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        hate_type_logits = self.hate_type_head(pooled_output)
        severity_logits = self.severity_head(pooled_output)
        to_whom_logits = self.to_whom_head(pooled_output)
        loss = None
        if labels is not None:
            hate_type_labels = labels[:, 0]
            severity_labels = labels[:, 1]
            to_whom_labels = labels[:, 2]
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(hate_type_logits, hate_type_labels.long()) + \
                   loss_fct(severity_logits, severity_labels.long()) + \
                   loss_fct(to_whom_logits, to_whom_labels.long())
        return SequenceClassifierOutput(
            loss=loss,
            logits=(hate_type_logits, severity_logits, to_whom_logits),
        )

non_label_column_names = [name for name in raw_datasets["combined"].column_names if name != "labels"]
sentence1_key= 'text'

# Padding strategy
padding = "max_length"

if max_seq_length > tokenizer.model_max_length:
    logger.warning(
        f"The max_seq_length passed ({max_seq_length}) is larger than the maximum length for the"
        f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}."
    )
max_seq_length = min(max_seq_length, tokenizer.model_max_length)

def preprocess_function(examples):
    # Tokenize the texts
    args = (
        (examples[sentence1_key],))
    result = tokenizer(*args, padding=padding, max_length=max_seq_length, truncation=True)
    if "hate_type" in examples:
        result["labels"] = [[l1, l2, l3] for l1, l2, l3 in zip(examples["hate_type"], examples["hate_severity"], examples["to_whom"])]
    return result

# Preprocess the datasets
raw_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    load_from_cache_file=True,
    desc="Running tokenizer on dataset",
)
# Prepare combined dataset for cross-validation
combined_dataset = raw_datasets["combined"]
predict_dataset = raw_datasets["test"]

# Extract features and labels for StratifiedKFold
# For stratification, use hate_type_label as primary, since it has more classes
X = np.arange(len(combined_dataset))  # Dummy, since we select indices
y = np.array(combined_dataset["hate_type"])  # Stratify on hate_type

print(f"Total samples for cross-validation: {len(y)}")
print(f"Label distribution: {np.bincount(y)}")

# Initialize StratifiedKFold
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Store results for each fold
fold_results = []
fold_probs = []
# Cross-validation loop
accuracy = evaluate.load("accuracy")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
def compute_metrics(p: EvalPrediction):
    hate_preds = np.argmax(p.predictions[0], axis=1)
    sev_preds = np.argmax(p.predictions[1], axis=1)
    to_preds = np.argmax(p.predictions[2], axis=1)
    hate_labels = p.label_ids[:,0]
    sev_labels = p.label_ids[:,1]
    to_labels = p.label_ids[:,2]
    hate_acc = accuracy.compute(predictions=hate_preds, references=hate_labels)['accuracy']
    sev_acc = accuracy.compute(predictions=sev_preds, references=sev_labels)['accuracy']
    to_acc = accuracy.compute(predictions=to_preds, references=to_labels)['accuracy']
    return {'hate_accuracy': hate_acc, 'severity_accuracy': sev_acc, 'to_whom_accuracy': to_acc}

class FGM():
    def __init__(self, model):
        self.model = model
        self.backup = {}

    def attack(self, epsilon=1.0, emb_name='base_model.embeddings.word_embeddings'):
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                self.backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                if norm != 0 and not torch.isnan(norm):
                    r_at = epsilon * param.grad / norm
                    param.data.add_(r_at)

    def restore(self, emb_name='base_model.embeddings.word_embeddings'):
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                if name in self.backup:
                    param.data = self.backup[name]
        self.backup = {}

class FADA():
    """Friendly Adversarial Data Augmentation
    Generates adversarial examples near decision boundary but not crossing it
    """
    def __init__(self, model, tokenizer, epsilon=0.3):
        self.model = model
        self.tokenizer = tokenizer
        self.epsilon = epsilon
        
    def generate_friendly_examples(self, inputs, model):
        """Generate friendly adversarial examples using word-level perturbations"""
        model.eval()
        with torch.no_grad():
            # Get original predictions
            outputs = model(**inputs)
            original_logits = outputs.logits
            
        # Apply small perturbation using FGM to get near-boundary examples
        model.train()
        inputs_copy = {k: v.clone() if torch.is_tensor(v) else v for k, v in inputs.items()}
        
        # Forward pass to get gradients
        outputs = model(**inputs_copy)
        loss = outputs.loss
        loss.backward()
        
        # Apply smaller epsilon for friendly adversarial examples (stay near boundary)
        fgm = FGM(model)
        fgm.attack(epsilon=self.epsilon, emb_name='base_model.embeddings.word_embeddings')
        
        # Get perturbed embeddings as friendly examples
        with torch.no_grad():
            friendly_outputs = model(**inputs_copy)
            
        fgm.restore(emb_name='base_model.embeddings.word_embeddings')
        
        return inputs_copy

class GATTrainer(Trainer):
    """Geometry-Aware Adversarial Training (GAT) Trainer
    Implements the GAT approach from the paper with FADA
    """
    def __init__(self, *args, use_gat=True, fada_epsilon=0.3, adv_epsilon=1.0, **kwargs):
        super().__init__(*args, **kwargs)
        self.use_gat = use_gat
        self.fada_epsilon = fada_epsilon
        self.adv_epsilon = adv_epsilon
        
    def training_step(self, model: torch.nn.Module, inputs: dict, num_items_in_batch: Optional[int] = None) -> torch.Tensor:
        model.train()
        inputs = self._prepare_inputs(inputs)
        
        if self.use_gat:
            # GAT approach: Clean -> Friendly Adversarial -> Strong Adversarial
            
            # Step 1: Forward pass on clean data
            with self.compute_loss_context_manager():
                loss_clean = self.compute_loss(model, inputs)
            
            if self.args.gradient_accumulation_steps > 1:
                loss_clean = loss_clean / self.args.gradient_accumulation_steps
            if self.args.n_gpu > 1:
                loss_clean = loss_clean.mean()
            
            # Backward pass for clean loss
            self.accelerator.backward(loss_clean)
            
            # Step 2: Generate friendly adversarial examples (near boundary)
            fgm_fada = FGM(model)
            fgm_fada.attack(epsilon=self.fada_epsilon, emb_name='base_model.embeddings.word_embeddings')
            
            with self.compute_loss_context_manager():
                loss_fada = self.compute_loss(model, inputs)
            
            if self.args.gradient_accumulation_steps > 1:
                loss_fada = loss_fada / self.args.gradient_accumulation_steps
            if self.args.n_gpu > 1:
                loss_fada = loss_fada.mean()
            
            # Backward pass for FADA loss
            self.accelerator.backward(loss_fada)
            fgm_fada.restore(emb_name='base_model.embeddings.word_embeddings')
            
            # Step 3: Strong adversarial training (key GAT contribution)
            fgm_adv = FGM(model)
            fgm_adv.attack(epsilon=self.adv_epsilon, emb_name='base_model.embeddings.word_embeddings')
            
            with self.compute_loss_context_manager():
                loss_adv = self.compute_loss(model, inputs)
            
            if self.args.gradient_accumulation_steps > 1:
                loss_adv = loss_adv / self.args.gradient_accumulation_steps
            if self.args.n_gpu > 1:
                loss_adv = loss_adv.mean()
            
            # Backward pass for adversarial loss
            self.accelerator.backward(loss_adv)
            fgm_adv.restore(emb_name='base_model.embeddings.word_embeddings')
            
            # Return average of all three losses
            return (loss_clean.detach() + loss_fada.detach() + loss_adv.detach()) / 3
            
        else:
            # Standard FGM adversarial training (fallback)
            # Step 1: Clean loss
            with self.compute_loss_context_manager():
                loss_clean = self.compute_loss(model, inputs)
            
            if self.args.gradient_accumulation_steps > 1:
                loss_clean = loss_clean / self.args.gradient_accumulation_steps
            if self.args.n_gpu > 1:
                loss_clean = loss_clean.mean()
            
            self.accelerator.backward(loss_clean)
            
            # Step 2: Adversarial loss
            fgm = FGM(model)
            fgm.attack(epsilon=self.adv_epsilon, emb_name='base_model.embeddings.word_embeddings')
            
            with self.compute_loss_context_manager():
                loss_adv = self.compute_loss(model, inputs)
            
            if self.args.gradient_accumulation_steps > 1:
                loss_adv = loss_adv / self.args.gradient_accumulation_steps
            if self.args.n_gpu > 1:
                loss_adv = loss_adv.mean()
            
            self.accelerator.backward(loss_adv)
            fgm.restore(emb_name='base_model.embeddings.word_embeddings')
            
            return (loss_clean.detach() + loss_adv.detach()) / 2

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"\n{'='*50}")
    print(f"FOLD {fold + 1}/{n_splits}")
    print(f"{'='*50}")
    
    # Create train and validation datasets for this fold
    train_dataset = combined_dataset.select(train_idx.tolist())
    val_dataset = combined_dataset.select(val_idx.tolist())
    
    # Remove ID columns
    train_dataset = train_dataset.remove_columns("id") if "id" in train_dataset.column_names else train_dataset
    val_dataset = val_dataset.remove_columns("id") if "id" in val_dataset.column_names else val_dataset
    
    print(f"Train size: {len(train_dataset)}, Validation size: {len(val_dataset)}")
    
    # Initialize model for this fold (fresh model each time)
    model = MultiTaskModel(model_name)
    
    # Update training arguments for this fold
    fold_training_args = TrainingArguments(
        learning_rate=2e-5,
        num_train_epochs=1,
        per_device_train_batch_size=8,  # Reduced for GAT memory requirements
        per_device_eval_batch_size=8,
        output_dir=f"./distilBERT_fold_{fold+1}/",
        overwrite_output_dir=True,
        remove_unused_columns=True,  # Changed to True to fix the tensor conversion error
        local_rank=1,
        load_best_model_at_end=True,
        save_total_limit=1,
        save_strategy="epoch",
        eval_strategy="epoch",
        logging_strategy="epoch",
        report_to=None,
        seed=42 + fold,  # Different seed for each fold
        gradient_accumulation_steps=2,  # Maintain effective batch size of 16
        fp16=True,  # Use mixed precision to save memory
    )
    
    # Initialize trainer for this fold with GAT
    trainer = GATTrainer(
        model=model,
        args=fold_training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
        data_collator=data_collator,
        use_gat=True,  # Enable GAT
        fada_epsilon=0.2,  # Reduced epsilon for memory efficiency
        adv_epsilon=0.5,  # Reduced epsilon for memory efficiency
    )
    
    # Train the model
    print(f"Training fold {fold + 1}...")
    train_result = trainer.train()
    
    # Evaluate the model
    print(f"Evaluating fold {fold + 1}...")
    eval_result = trainer.evaluate()
    
    # Store results
    fold_results.append({
        'fold': fold + 1,
        'train_loss': train_result.metrics['train_loss'],
        'eval_loss': eval_result['eval_loss'],
        'eval_hate_accuracy': eval_result['eval_hate_accuracy'],
        'eval_severity_accuracy': eval_result['eval_severity_accuracy'],
        'eval_to_whom_accuracy': eval_result['eval_to_whom_accuracy']
    })
    
    # Generate predictions on test set for this fold
    print(f"Predicting with fold {fold + 1} model...")
    test_predictions = trainer.predict(predict_dataset.remove_columns("id") if "id" in predict_dataset.column_names else predict_dataset)
    probs = [torch.softmax(torch.tensor(logits), dim=-1).numpy() for logits in test_predictions.predictions]
    fold_probs.append(probs)
    
    # Clean up to save memory
    del model, trainer
    
    print(f"Fold {fold + 1} - Hate Accuracy: {eval_result['eval_hate_accuracy']:.4f}, Severity Accuracy: {eval_result['eval_severity_accuracy']:.4f}, To Whom Accuracy: {eval_result['eval_to_whom_accuracy']:.4f}")
    
print(f"\n{'='*50}")
print("CROSS-VALIDATION COMPLETED")
print(f"{'='*50}")
# Analyze cross-validation results
import pandas as pd

results_df = pd.DataFrame(fold_results)
print("\nCross-Validation Results:")
print(results_df)

# Calculate average performance metrics
avg_train_loss = results_df['train_loss'].mean()
avg_eval_loss = results_df['eval_loss'].mean()
avg_hate_acc = results_df['eval_hate_accuracy'].mean()
avg_sev_acc = results_df['eval_severity_accuracy'].mean()
avg_to_acc = results_df['eval_to_whom_accuracy'].mean()
std_hate_acc = results_df['eval_hate_accuracy'].std()
std_sev_acc = results_df['eval_severity_accuracy'].std()
std_to_acc = results_df['eval_to_whom_accuracy'].std()

print(f"\nAverage Results Across {n_splits} Folds:")
print(f"Average Training Loss: {avg_train_loss:.4f}")
print(f"Average Validation Loss: {avg_eval_loss:.4f}")
print(f"Average Hate Type Accuracy: {avg_hate_acc:.4f} ± {std_hate_acc:.4f}")
print(f"Average Severity Accuracy: {avg_sev_acc:.4f} ± {std_sev_acc:.4f}")
print(f"Average To Whom Accuracy: {avg_to_acc:.4f} ± {std_to_acc:.4f}")

# Ensemble predictions
hate_probs_folds = np.array([probs[0] for probs in fold_probs])
sev_probs_folds = np.array([probs[1] for probs in fold_probs])
to_probs_folds = np.array([probs[2] for probs in fold_probs])
ensemble_probs = (np.mean(hate_probs_folds, axis=0),
                  np.mean(sev_probs_folds, axis=0),
                  np.mean(to_probs_folds, axis=0))
# np.save('ensemble_probs_aug20.npy', np.array(ensemble_probs, dtype=object))
# Final ensemble prediction
hate_probs, sev_probs, to_probs = ensemble_probs
final_hate_preds = np.argmax(hate_probs, axis=1)
final_sev_preds = np.argmax(sev_probs, axis=1)
final_to_preds = np.argmax(to_probs, axis=1)

# Generate predictions


# # Also save the ensemble predictions with different format for comparison
# submission_df = pd.DataFrame({'id': test_df['id'], 'hate_type': [id2hate[p] for p in final_hate_preds], 'hate_severity': [id2sev[p] for p in final_sev_preds], 'to_whom': [id2to[p] for p in final_to_preds]})
# submission_df.to_csv('ensemble_submission.tsv', sep='\t', index=False)

# print("Ensemble predictions also saved to 'ensemble_submission.tsv'")
import os
os.makedirs(training_args.output_dir, exist_ok=True)
logger.info("*** Predict ***")
ids = test_df['id']
output_predict_file = os.path.join(training_args.output_dir, f"subtask_1C.tsv")

# Write predictions in the required format
with open(output_predict_file, "w") as writer:
    logger.info(f"***** Predict results *****")
    writer.write("id\thate_type\thate_severity\tto_whom\tmodel\n")
    for index in range(len(final_hate_preds)):
        h = id2hate[final_hate_preds[index]]
        s = id2sev[final_sev_preds[index]]
        t = id2to[final_to_preds[index]]
        writer.write(f"{ids[index]}\t{h}\t{s}\t{t}\t{model_name}\n")

print(f"\nPredictions saved to '{output_predict_file}'")

--2025-08-26 14:27:37--  https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1C/blp25_hatespeech_subtask_1C_dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 634005 (619K) [text/plain]
Saving to: ‘blp25_hatespeech_subtask_1C_dev.tsv’

blp25_hatespeech_su 100%[===================>] 619.15K  --.-KB/s    in 0.04s   

2025-08-26 14:27:38 (14.1 MB/s) - ‘blp25_hatespeech_subtask_1C_dev.tsv’ saved [634005/634005]

--2025-08-26 14:27:38--  https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1C/blp25_hatespeech_subtask_1C_dev_test.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (r

2025-08-26 14:29:15.385186: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756218555.584792      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756218555.643948      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Combined dataset size: 38034
Test dataset size: 2512


tokenizer_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/586 [00:00<?, ?B/s]

...

[INFO|trainer.py:934] 2025-08-26 21:36:28,927 >> The following columns in the Evaluation set don't have a corresponding argument in `MultiTaskModel.forward` and have been ignored: hate_type, hate_severity, text, to_whom, token_type_ids. If hate_type, hate_severity, text, to_whom, token_type_ids are not expected by `MultiTaskModel.forward`,  you can safely ignore this message.
[INFO|trainer.py:4327] 2025-08-26 21:36:28,932 >> 
***** Running Evaluation *****
[INFO|trainer.py:4329] 2025-08-26 21:36:28,932 >>   Num examples = 7606
[INFO|trainer.py:4332] 2025-08-26 21:36:28,933 >>   Batch size = 16
[INFO|trainer.py:3993] 2025-08-26 21:39:09,047 >> Saving model checkpoint to ./distilBERT_fold_5/checkpoint-951
[INFO|trainer.py:4007] 2025-08-26 21:39:09,050 >> Trainer.model is not a `PreTrainedModel`, only saving its state dict.
[INFO|tokenization_utils_base.py:2525] 2025-08-26 21:39:09,846 >> tokenizer config file saved in ./distilBERT_fold_5/checkpoint-951/tokenizer_config.json
[INFO|tokeniz

Evaluating fold 5...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


[INFO|trainer.py:934] 2025-08-26 21:41:51,540 >> The following columns in the test set don't have a corresponding argument in `MultiTaskModel.forward` and have been ignored: token_type_ids, text. If token_type_ids, text are not expected by `MultiTaskModel.forward`,  you can safely ignore this message.
[INFO|trainer.py:4327] 2025-08-26 21:41:51,544 >> 
***** Running Prediction *****
[INFO|trainer.py:4329] 2025-08-26 21:41:51,545 >>   Num examples = 2512
[INFO|trainer.py:4332] 2025-08-26 21:41:51,545 >>   Batch size = 16


Predicting with fold 5 model...
Fold 5 - Hate Accuracy: 0.7144, Severity Accuracy: 0.7491, To Whom Accuracy: 0.7139

CROSS-VALIDATION COMPLETED

Cross-Validation Results:
   fold  train_loss  eval_loss  eval_hate_accuracy  eval_severity_accuracy  \
0     1    2.573692   2.153476            0.707243                0.747207   
1     2    2.577549   2.150924            0.714079                0.741817   
2     3    2.552672   2.160430            0.707769                0.735112   
3     4    2.567124   2.161605            0.708164                0.746549   
4     5    2.569832   2.142720            0.714436                0.749145   

   eval_to_whom_accuracy  
0               0.709610  
1               0.710398  
2               0.712502  
3               0.708426  
4               0.713910  

Average Results Across 5 Folds:
Average Training Loss: 2.5682
Average Validation Loss: 2.1538
Average Hate Type Accuracy: 0.7103 ± 0.0036
Average Severity Accuracy: 0.7440 ± 0.0056
Average To Whom 